# **Alineamiento múltiple de secuencias** 
## Software utilizado: BLAST y MAFFT

Inicialmente importamos las librerías necesarias para ejecutar con éxito el alineamiento múltiple

Posteriormente buscamos datos como secuencia y nombre de GLUT1 en base al código de indexación en la base de datos UNIPROT y lo guardamos en la carpeta *"../run"*

In [12]:
# Libraries import
from Bio import ExPASy, SwissProt
from Bio import SeqIO
from Bio import AlignIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.PDB import PDBList
import os
import xml.etree.ElementTree as ET
from io import StringIO
import subprocess
import pandas as pd
import requests

In [ ]:
uniprot_code = 'P11166' # GLUT1 - SLC2A1

#=== Fetch the sequence from UniProt ===
handle = ExPASy.get_sprot_raw(uniprot_code)
record = SeqIO.read(handle, 'swiss')
prot_nm = record.name
prot_sq = record.seq

#=== Save as FASTA file ===
output_dir = "../run/"
os.makedirs(output_dir, exist_ok=True)
fasta_path = os.path.join(output_dir, f"{prot_nm}.fasta")

with open(fasta_path, 'w') as fasta_file:
    SeqIO.write(record, fasta_file, 'fasta')

#=== Print the sequence ===
print(f'Nombre de la proteína: ', prot_nm)
print(f"Archivo FASTA guardado en: {fasta_path}")

Si conoces el PDB_ID de la o las proteinas que quieres alinear, ejecuta esta celda.

Si no es así, ejecuta la celda siguiente

In [16]:
def download_fasta_from_pdb(pdb_ids, outputdir):
    """
    Descarga secuencias FASTA para una lista de IDs de PDB dados.
    """
    base_url = "https://www.uniprot.org/uploadlists/"
    
    # Prepara los IDs para la petición. UniProt puede manejar múltiples IDs separados por espacio o coma.
    # Usaremos el formato UniProt (mapeo de PDB a UniProt)
    
    # La interfaz de UniProt permite mapear PDB IDs a UniProt IDs y luego obtener los FASTA.
    # Otra opción más directa, si solo necesitas la secuencia de la entrada PDB específica
    # y no el mapeo a UniProt, es usar el RCSB PDB directamente.

    print("Descargando archivos FASTA...")
    
    for pdb_id in pdb_ids:
        # Usaremos el servidor del RCSB PDB directamente para obtener el FASTA de la entrada PDB.
        # Esto suele ser más directo para secuencias asociadas con la entrada PDB.
        fasta_url = f"https://www.rcsb.org/fasta/entry/{pdb_id}/download"
        os.makedirs(output_dir, exist_ok=True)

        try:
            response = requests.get(fasta_url)
            response.raise_for_status()  # Genera una excepción para errores HTTP (4xx o 5xx)
            
            # El contenido de la respuesta es el archivo FASTA
            fasta_content = response.text
            
            # Guarda el contenido en un archivo FASTA
            file_name = f"{pdb_id}.fasta"
            fullpath = os.path.join(output_dir, file_name)
            
            with open(fullpath, 'w') as f:
                f.write(fasta_content)
            
            print(f"✔️ {file_name} descargado exitosamente.")
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Error al descargar {pdb_id}: {e}")
        except Exception as e:
            print(f"❌ Ocurrió un error inesperado con {pdb_id}: {e}")

# --- Uso del código ---
if __name__ == "__main__":
    # Puedes cambiar esta lista por los IDs de PDB que necesites
    pdb_identifiers = ['4pyp', '5eqg', '5eqh', '5eqi', '6tha'] 
    output_dir = "../run/comparativa"

    download_fasta_from_pdb(pdb_identifiers, output_dir)

Descargando archivos FASTA...
✔️ 4pyp.fasta descargado exitosamente.
✔️ 5eqg.fasta descargado exitosamente.
✔️ 5eqh.fasta descargado exitosamente.
✔️ 5eqi.fasta descargado exitosamente.
✔️ 6tha.fasta descargado exitosamente.


Una vez obtenidos esos datos, creamos una subcarpeta para guardar nuestros alineamientos.
El fin de realizar este paso es identificar posibles homólogos de las secuencias alineadas en base a nuestra proteína, lo que puede proporcionar información valiosa sobre su función, origen evolutivo y relaciones funcionales. 

Al ejecutar BLAST, se genera un archivo *.xml*, el cual guardaremos en la carpeta recíen creada

In [ ]:
# === General settings ===
output_dir = "../run/blast_mafft"
os.makedirs(output_dir, exist_ok=True)

xml_path = os.path.join(output_dir, "blast_results.xml")
fasta_path = os.path.join(output_dir, "hits_blast.fasta")
aligned_path = os.path.join(output_dir, "alignment_mafft.fasta")

#=== Run BLASTp online ===
print("Ejecutando BLASTp...")
try:
    result = NCBIWWW.qblast('blastp', 'pdb', prot_sq, hitlist_size=10)
    with open(xml_path, 'w') as out_xml:
        out_xml.write(result.read())
    result.close()
    print(f'Resultado BLAST guardado en {xml_path}')
except Exception as ex:
    print("Error durante BLAST:", ex)
    exit()

Luego, de los datos obtenidos, los analizaremos para facilitar la redacción y extracción de información importante como lo es la secuencia de cada proteína obtenida en el proceso anterior

In [ ]:
#=== Parse results and extract sequences ===
print("Extrayendo secuencias...")
records = []
with open(xml_path) as result_file:
    blast_record = NCBIXML.read(result_file)

i = 1
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.sbjct and isinstance(hsp.sbjct, str):
            clean_seq = hsp.sbjct.replace("-", "").strip()
            seq = Seq(clean_seq)
            rec_id = f'{alignment.accession}_{i}'
            description = alignment.hit_def
            record = SeqRecord(seq, id=rec_id, description=description)
            records.append(record)
            i += 1
        break
    if i > 10:
        break

if not records: # Sequence validation
    print("No se extrajeron secuencias válidas")
    exit()

SeqIO.write(records, fasta_path, "fasta") # Save in FASTA format
print(f"{len(records)} secuencias guardadas en: {fasta_path}")


A continuación se ejecutará el alineamiento multiple de secuencias con MAFFT. Dentro del código se crean directorios para alojar los outputs (.aln y .fasta) generados del alineamiento y se genera una tabla para visualizar el alineamiento y confirmar su correcta ejecución 

In [ ]:
def run_mafft(fasta_path: str, output_dir: str, base_name: str = "MSA"):
    print("Ejecutando alineamiento múltiple con MAFFT")

    os.makedirs(output_dir, exist_ok=True) # confirm directory

    try:
        result = subprocess.run(
            ["mafft", "--auto", fasta_path],
            capture_output=True,
            text=True,
            check=True
        )
    except subprocess.CalledProcessError as er:
        print("Error al ejecutar MAFFT:")
        print(er.stderr)
        return None
    
    # Parsing alignment
    alignment = AlignIO.read(StringIO(result.stdout), 'fasta')

    # Save formats
    fasta_out = os.path.join(output_dir, f'{base_name}.fasta')
    clustal_out = os.path.join(output_dir, f'{base_name}.aln')

    AlignIO.write(alignment, fasta_out, "fasta")
    AlignIO.write(alignment, clustal_out, "clustal")

    print(f'\n Alineamiento guardado en:')
    print(f'    * FASTA     : {fasta_out}')
    print(f'    * CLUSTAL   : {clustal_out}')

    return alignment # for visualize

#=== Running alignment ===
output_dir = '../run/blast_mafft'

alignment = run_mafft(fasta_path, output_dir)       # For execution


#=== Visualize the alignment as a table===
if alignment:
    print("\n Vista tabular del alineamiento")
    df = pd.DataFrame({
        record.id: list(str(record.seq)) for record in alignment
    }).T        # Show each sequence in rows
    df.columns = [f'Pos {i+1}'for i in range(df.shape[1])]
    display(df)